# 初期準備

In [ ]:
!pip install beautifulsoup4

In [ ]:
import requests as req
import re
import os
from os import path
from bs4 import BeautifulSoup
import time

In [ ]:
session = req.Session()

In [ ]:
# 出力先フォルダ
out_dir = 'C:/temp/qrunch'

# function

In [ ]:
def qrunch_export_md(entrie_url, out_dir):
    # ダッシュボードにアクセス
    res = session.get(entrie_url)

    # parse
    soup = BeautifulSoup(res.text, 'lxml')

    # markdownのテキストを取得する
    file_name = soup.find('input', class_='title-input').get('value').replace(' ', '_') + '.md'
    md_text = soup.find_all('textarea', id='edit-box')[0].contents[0]
    
    # ファイルの書き出し
    with open(path.join(out_dir, file_name), mode='w', encoding='utf8') as f:
        f.writelines(md_text)
    
    # これより上の階層だと忘れそうだったのでここにスリープ入れちゃう
    time.sleep(1)

# Twitterログイン
[Twitter ログインのウェブサイトをスクレイピングする](https://qiita.com/sqrtxx/items/ea76209825084c3938ce) を参照しました。

In [ ]:
# ログイン
res = session.get('https://qrunch.net/users/auth/twitter')
soup = BeautifulSoup(res.text.encode(res.encoding), 'lxml')
oauth_token = soup.find(attrs={'id': 'oauth_token'}).get('value')
authenticity_token = soup.find(attrs={'name':'authenticity_token'}).get('value')
redirect_after_login = soup.find(attrs={'name':'redirect_after_login'}).get('value')

twitter_id = 'xxx'
twitter_password = 'xxx'

login_info = {
    'authenticity_token': authenticity_token,
    'redirect_after_login': redirect_after_login,
    'oauth_token': oauth_token,
    'session[username_or_email]': twitter_id,
    'session[password]': twitter_password
}

twitter_oauth_url = "https://twitter.com/oauth/authorize"
login_res = session.post(twitter_oauth_url, data=login_info)
login_soup = BeautifulSoup(login_res.text.encode(login_res.encoding), 'lxml')
redirect_to = login_soup.select('.maintain-context')[0]['href']

session.get(redirect_to)

# 記事のエクスポート

In [ ]:
# ダッシュボードにアクセス
res = session.get('https://qrunch.net/dashboard/entries')

# parse
soup = BeautifulSoup(res.text, 'lxml')

# 記事一覧の取得
entries = []
for elems in soup.find_all('a', href=re.compile('/dashboard/entries/.+edit')):
    link = 'https://qrunch.net{}'.format(elems.attrs['href'])
    
    print(link)
    entries.append(link)

In [ ]:
# 出力先フォルダの作成
entries_dir = path.join(out_dir, 'entries')
if not path.exists(entries_dir):
    os.mkdir(entries_dir)

# 出力
for entrie_link in entries:
    qrunch_export_md(entrie_link, entries_dir)

# ログのエクスポート

In [ ]:
# ダッシュボードにアクセス
res = session.get('https://qrunch.net/dashboard/logs')

# parse
soup = BeautifulSoup(res.text, 'lxml')

# ログ一覧の取得
logs = []
for elems in soup.find_all('a', href=re.compile('/dashboard/logs/.+edit')):
    link = 'https://qrunch.net{}'.format(elems.attrs['href'])
    
    print(link)
    logs.append(link)

In [ ]:
# 出力先フォルダの作成
log_dir = path.join(out_dir, 'logs')
if not path.exists(log_dir):
    os.mkdir(log_dir)

# 出力
for log_link in logs:
    qrunch_export_md(log_link, log_dir)